### 1. 问题
#### 【问题一】 什么是fillna的前向/后向填充，如何实现？

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv',index_col='ID')
df.loc[1102, ['Height', 'Weight']] = np.nan
#后向填充，用缺失行后面最近的一行有效记录的值填充
df.fillna(method='bfill').head()
#前向填充，用缺失行前面最近的一行有效记录的值填充
df.fillna(method='ffill').head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173.0,63.0,34.0,A+
1102,S_1,C_1,F,street_2,173.0,63.0,32.5,B+
1103,S_1,C_1,M,street_2,186.0,82.0,87.2,B+
1104,S_1,C_1,F,street_2,167.0,81.0,80.4,B-
1105,S_1,C_1,F,street_4,159.0,64.0,84.8,B+


#### 【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。

In [6]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()
result = s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1,fill_value =0)

#上述的代码实现了以0为分界符的计数器

l = [0, 1, 1, 0, 1, 1, 1, 0]
new_s = pd.Series(l, index=l)
group_id = 0
def group_split(x):
    global group_id
    if x == 0:  #新的分组
        group_id = group_id + 1
    return('group' + str(group_id))
new_s.groupby(group_split).cumsum()

0    0
1    1
1    2
0    0
1    1
1    2
1    3
0    0
dtype: int64

#### 【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。

In [61]:
df = pd.read_csv('data/table.csv',index_col='ID')
df.groupby('School')['Math'].quantile([0.25, 0.75])

School      
S_1     0.25    41.850
        0.75    85.000
S_2     0.25    47.500
        0.75    72.225
Name: Math, dtype: float64

#### 【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？

#### 【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？

In [28]:
#Aggregation输入Series，输出标量，用于获得统计数据
df = pd.read_csv('data/table.csv',index_col='ID')
def agg_func(s):
    print(s.head())
#df.groupby('School').agg(agg_func)

#Transformation输入Series，输出同长度的Series，用于对Series进行变换
def trans_func(s):
    print(type(s))
#df.groupby('School').transform(trans_func)

#Filteration输入分组后的DataFrame，要么原样输出要么不输出，用于按条件对DataFrame进行过滤
def filt_func(s):
    print(type(s))
df.groupby('School').filter(filt_func)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,


#### 【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？

In [55]:
df = pd.read_csv('data/table.csv',index_col='ID')

def count_between(s, low, high):
    return(s[s.between(low,high)].count())
count_between_50_100 = lambda x:count_between(x, 50, 100)

df.groupby('School').agg([count_between_50_100, 'mean']).rename(columns={'<lambda_0>':'count_between_50_100'})

Height                           Weight         \
       count_between_50_100        mean count_between_50_100   mean   
School                                                                
S_1                       0  175.733333                   15  70.00   
S_2                       0  172.950000                   20  78.15   

                       Math             
       count_between_50_100       mean  
School                                  
S_1                    10.0  63.746667  
S_2                    12.0  59.555000

### 2. 练习
#### 【练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：
#### (a) 在所有重量超过1克拉的钻石中，价格的极差是多少？

In [9]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/Diamonds.csv')
df_price = df[df['carat'] > 1]['price']
df_price.max() - df_price.min()

17561

#### (b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？

In [117]:
group_depth = df.groupby(pd.qcut(df['depth'], 5))
#方法1
group_depth['color'].agg(lambda x:x.value_counts().index[0])
#方法2
group_depth['color'].describe()

#方法1
df['unit_price'] = df['price'] / df['carat']
def func(sub_df):
    group_mul = sub_df.groupby('color')
    return(group_mul['unit_price'].mean().idxmax())
group_depth.apply(func)
#方法2
group_mul = df.groupby([pd.qcut(df['depth'], 5), 'color'])
[i[1] for i in group_mul['unit_price'].mean().groupby('depth').idxmax().values]

['I', 'F', 'F', 'G', 'G']

#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。

In [42]:
group_carat = df.groupby(pd.cut(df['carat'], bins=[0, 0.5, 1, 1.5, 2, df['carat'].max()]))
def count_continus(l):
    l1 = []
    count = 0
    pre = 0
    for i in l:
        if i > pre:
            count += 1
        else:
            count = 1
        pre = i
        l1.append(count)
    return(max(l1))
def func(sub_df):
    #方法1
    price_values = sub_df[['depth', 'price']].set_index('depth').sort_index()['price'].tolist()
    #方法2
    #price_values = sub_df.sort_values(by='depth')['price'].tolist()
    return(count_continus(price_values))
group_carat.apply(func)

carat
(0.0, 0.5]      8
(0.5, 1.0]      8
(1.0, 1.5]      7
(1.5, 2.0]     11
(2.0, 5.01]     7
dtype: int64

#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

#### 【练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：
#### (a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？

In [71]:
df = pd.read_csv('data/Drugs.csv')
df.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [98]:
s_state = df[['COUNTY', 'State']].drop_duplicates().set_index('COUNTY')['State']
group_year = df.groupby('YYYY')
for name, group in group_year:
    top_county_in_year = group.groupby('COUNTY')['DrugReports'].sum().idxmax()
    top_state_in_year = group.groupby('State')['DrugReports'].sum().idxmax()
    print(name, top_county_in_year, s_state[top_county_in_year] == top_state_in_year)

2010 PHILADELPHIA True
2011 PHILADELPHIA False
2012 PHILADELPHIA False
2013 PHILADELPHIA False
2014 PHILADELPHIA False
2015 PHILADELPHIA False
2016 HAMILTON True
2017 HAMILTON True


#### (b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。

In [179]:
s_h = df[(df['YYYY'].isin([2014,2015])) & (df['SubstanceName'] == 'Heroin')].groupby(['YYYY', 'State'])['DrugReports'].sum()
(s_h[2015] - s_h[2014]).idxmax()

'OH'

In [185]:
s_d = df.query('(YYYY in [2014, 2015]) and (State == "OH")').groupby(['YYYY', 'SubstanceName'])['DrugReports'].sum()
((s_d[2015] - s_d[2014]) / s_d[2014]).idxmax()

'Acetyl fentanyl'